In [1]:
import requests
import json
import pandas as pd
import datetime, time
from io import BytesIO
import zipfile
import os
import mysql.connector
from sqlalchemy import create_engine
import glob

url = "https://api.ringba.com/v2/token"

payload='grant_type=password&username=mike%40carida.com&password=Rutgers-72'
headers = {
  'content-type': 'application/x-www-form-urlencoded; charset=UTF-8'
}

response = requests.request("POST", url, headers=headers, data=payload)

token = response.json()
token = token['access_token']

today = datetime.date.today()

# Calculate yesterday
yesterday = today - datetime.timedelta(days=1)

# If today is Monday (weekday() function returns 0 for Monday) then set yesterday to be last Friday
if today.weekday() == 0:
    yesterday = today - datetime.timedelta(days=3)

# Convert yeserday variable to string
yesterday = str(yesterday)

# This section is not required anymore as you're not using 'day' and 'today' variables anywhere
# day = datetime.date.today()
# today = time.strftime("%Y-%m-%d")

# Calculate start and end time for the given day
start = yesterday + 'T' + '00:00:00Z'
end = yesterday + 'T' + '23:59:59Z'

In [2]:
print(start,end)

2023-06-07T00:00:00Z 2023-06-07T23:59:59Z


In [3]:
url = "https://api.ringba.com/v2/RA0909aa9eddae40c4bb48f7253e6e8742/calllogs/export/csv"

payload = json.dumps({
  "reportEnd": end,
  "reportStart": start,
  "orderByColumns": [
      {
          "column": "callDt",
          "direction": "desc"
      }
  ],
  "valueColumns": [
    {
      "column": "callDt"
    },
    {
      "column": "tag:DialedNumber:Name"
    },
    {
      "column": "campaignName"
    },
    {
      "column": "publisherName"
    },
    {
      "column": "targetName"
    },
    {
      "column": "targetNumber"
    },
    {
      "column": "buyer"
    },
    {
      "column": "inboundPhoneNumber"
    },
    {
      "column": "number"
    },
    {
      "column": "endCallSource"
    },
    {
      "column": "hasConnected"
    },
    {
      "column": "callLengthInSeconds"
    },
    {
      "column": "connectedCallLengthInSeconds"
    },
    {
      "column": "hasPayout"
    },
    {
      "column": "isDuplicate"
    },
    {
      "column": "hasPreviouslyConnected"
    },
    {
      "column": "hasConverted"
    },
    {
      "column": "payoutAmount"
    },
    {
      "column": "tag:InboundNumber:State"
    },
  ]
})
headers = {
  'Authorization': 'Bearer' + ' ' + token,
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

csvid = response.json()['id']

In [4]:
url = "https://api.ringba.com/v2/RA0909aa9eddae40c4bb48f7253e6e8742/calllogs/export/" + csvid
payload={}
headers = {
  'Authorization': 'Bearer' + ' ' + token
  }
response = requests.request("GET", url, headers=headers, data=payload)
#time.sleep(420)
r = response.json()

In [5]:
'''poll the API until the file is ready'''
while True:
    response = requests.request("GET", url, headers=headers, data=payload)
    if response.json()['status'] == 'Ready':
        break
    else:
        time.sleep(5)
        continue
# response = requests.request("GET", url, headers=headers, data=payload)
r = response.json()['url']

print('Downloading started')

filename = url.split('/')[-1]

req = requests.get(r)
print('Downloading finished')

'''extract the file to the current directory'''
with zipfile.ZipFile(BytesIO(req.content)) as zip_ref:
    zip_ref.extractall()

In [6]:
df = pd.concat(map(pd.read_csv, glob.glob('ringba-call-log*.csv')))

In [7]:
# Step: Change data type of ['Campaign', 'Publisher', 'Target', 'Buyer', 'End Call Source', 'Connected', 'Paid Out', 'Is Duplicate', 'Converted'] to String/Text
for column_name in df.select_dtypes(['object']).columns:
    df[column_name] = df[column_name].astype('string')

# Step: Change data type of ['Caller ID', 'Number'] to String/Text
for column_name in ['Caller ID', 'Number']:
    df[column_name] = df[column_name].astype('string')

# Step: Change data type of ['Call Length', 'Connected Call Length', 'Payout'] to Integer
for column_name in df.select_dtypes('float').columns:
    df[column_name] = df[column_name].astype('Int64')

# Step: Drop rows where Payout is missing
df = df.loc[~(df['Payout'].isna())]

df['Call Date'] = pd.to_datetime(df['Call Date'], unit='ms')

df = df.drop(columns=['Converted', 'Campaign'])


In [8]:
# Step: Keep rows where Publisher is one of: PX Media
PX = df.loc[df['Publisher'].isin(['Wecall Media'])]

In [9]:
sql_str = 'mariadb+pymysql://mike@data.caridamedsupp.com:Rutgers-72@45.79.131.179/Customers'
          
# Create the connection
cnx = create_engine(sql_str)

PX.to_sql('WeCall Media Ringba Call Log', con=cnx, if_exists='append', index=False)

71

In [10]:
'''delete the file that begins with ringba-call-log'''
for file in glob.glob('ringba-call-log*.csv'):
    os.remove(file)

In [11]:
auth_token = "vu332i5d8jx6ppqyhmkq76sxs2e2w6fh"
start_time = start
end_time = end
limit = 500  # You can use any value between 1 and 500 for the limit parameter
offset = 0  # Set the offset to 0 to start pagination from the beginning
order = "asc"  # Set the order to "asc" to retrieve the data in ascending order
call_type =  '' # Set the call_type to "inbound" to retrieve only inbound call data
called_count = '' # Set the called_count to 0 to retrieve only calls that were not answered
include_recordings = 0  # Set the include_recordings to 0 to exclude recordings from the response
list_id = 2990

In [12]:
def convoso_api_to_dataframe(auth_token, start_time, end_time, limit, offset, order, call_type, called_count, include_recordings):
    url = f"https://api.convoso.com/v1/log/retrieve?auth_token={auth_token}&status=&list_id={list_id}&start_time={start_time}&end_time={end_time}&limit={limit}&offset={offset}&order={order}&call_type={call_type}&called_count={called_count}&include_recordings={include_recordings}"
    response = requests.get(url)
    data = response.json()['data']['results']
    df = pd.json_normalize(data)
    return df


In [13]:
df

,Call Date,tag:DialedNumber:Name,Publisher,Target,Target Number,Buyer,Caller ID,Number,End Call Source,Connected,Call Length,Connected Call Length,Paid Out,Is Duplicate,Previously Connected,Payout,tag:InboundNumber:State
1,2023-06-07 22:29:50.168,Wecall Media - MA,Wecall Media,Wecall Media,18336103308,carida_ms,15735288857,18885731230,Target,True,107,104,True,False,False,40,MO
6,2023-06-07 22:26:50.234,MediaAlpha Medicare,MediaAlpha,"MediaAlpha - Datalot, etc (Insured)_Convoso",18337023240,carida_ms,13474234869,18665131157,Caller,True,168,164,True,False,False,40,NY
11,2023-06-07 22:21:56.159,Wecall Media - MA,Wecall Media,Wecall Media,18336103308,carida_ms,13134627371,18885731230,Target,True,113,109,True,False,False,40,MI
12,2023-06-07 22:18:18.951,Wecall Media - MA,Wecall Media,Wecall Media,18336103308,carida_ms,12164100553,18885731230,Caller,True,146,142,True,False,False,40,OH
14,2023-06-07 22:15:52.135,MediaAlpha Medicare,MediaAlpha,"MediaAlpha - Datalot, etc (Insured)_Convoso",18337023240,carida_ms,15809547665,18665131157,Target,True,1261,1257,True,False,True,40,OK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
775,2023-06-07 14:02:35.541,Medialot Tier 1,Medialot,Medialot,18334202566,carida_ms,17798033133,18663861361,Target,True,171,167,True,False,False,52,IL
777,2023-06-07 14:01:13.297,PolicyBind,PolicyBind,PolicyBind,18334202703,carida_ms,18502123185,18447591239,Target,True,663,659,True,False,False,45,FL
779,2023-06-07 14:00:32.686,PolicyBind,PolicyBind,PolicyBind,18334202703,carida_ms,12819952792,18447591239,Caller,True,163,158,True,False,False,45,TX
780,2023-06-07 14:00:27.630,PX MA Calls,PX Media,PX - MA,18337382431,carida_ms,19294983399,18886591371,Caller,True,3653,3649,True,False,False,40,NY


In [14]:
# Create an empty dataframe to store the concatenated data
df_all = pd.DataFrame()

# Iterate over a range of values for the offset parameter
for offset in range(0, 40000, 500):
    # Call the function and store the returned dataframe in a variable
    df = convoso_api_to_dataframe(auth_token, start_time, end_time, limit, offset, order, call_type, called_count, include_recordings)
    
    # Concatenate the dataframe with the df_all dataframe
    df_all = pd.concat([df_all, df], ignore_index=True)

In [15]:
df_all

,id,lead_id,list_id,campaign_id,campaign,queue,user,user_id,phone_number,first_name,...,agent_comment,queue_id,called_count,caller_id_displayed,inbound_number,term_reason,call_type,queue_position,queue_seconds,originating_agent_id
0,2836366,606194,2990,173,CMO Campaign,Carida 360_alt_footprint,Andrea Adler,1126835,5735288857,None,...,None,1728,0,18336103308,18336103308,AGENT,INBOUND,0,1.00,None
1,2836365,606193,2990,173,CMO Campaign,Carida 360_alt_footprint,System DID User,666667,9144506875,None,...,None,1728,0,18336103308,18336103308,ABANDON,INBOUND,0,18.00,None
2,2836362,606190,2990,173,CMO Campaign,Carida 360_alt_footprint,System DID User,666667,9297635297,None,...,None,1728,0,18336103308,18336103308,ABANDON,INBOUND,0,1.00,None
3,2836360,606188,2990,173,CMO Campaign,Carida 360_alt_footprint,System DID User,666667,9155887834,None,...,None,1728,0,18336103308,18336103308,ABANDON,INBOUND,0,16.00,None
4,2836357,606187,2990,173,CMO Campaign,Carida 360_alt_footprint,System DID User,666667,7707127186,None,...,None,1728,0,18336103308,18336103308,QUEUETIMEOUT,INBOUND,0,70.00,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,2830660,604777,2990,173,CMO Campaign,Carida 360_alt_footprint,Jaqueline Dulin,1126754,7042682752,Marcia,...,,1728,0,18336103308,18336103308,AGENT,INBOUND,0,0.00,None
211,2830656,604777,2990,173,CMO Campaign,Carida 360_alt_footprint,Christopher Hamersma,1126825,7042682752,Marcia,...,,1728,0,18336103308,18336103308,CALLER,INBOUND,0,0.00,None
212,2830652,604775,2990,173,CMO Campaign,Carida 360_alt_footprint,Dennis Blake,1130036,9033377747,,...,,1728,0,18336103308,18336103308,CALLER,INBOUND,0,0.00,None
213,2830646,604772,2990,173,CMO Campaign,Carida 360_alt_footprint,Vivian Gross,1107312,2197027276,sylvia,...,None,1728,0,18336103308,18336103308,CALLER,INBOUND,0,0.00,None


In [16]:
import pandas as pd; import numpy as np
# Step: Keep rows where list_id is one of: 2907
df_all = df_all.loc[df_all['list_id'].isin(['2990'])]

import pandas as pd; import numpy as np
# Step: Select columns
df_all = df_all[['phone_number', 'status_name', 'call_date']]

# Step: Change data type of call_date to Datetime
df_all['call_date'] = pd.to_datetime(df_all['call_date'], infer_datetime_format=True)

# Step: Change data type of status_name to String/Text
df_all['status_name'] = df_all['status_name'].astype('string')

# Step: Change data type of phone_number to String/Text
df_all['phone_number'] = df_all['phone_number'].astype('string')

import pandas as pd; import numpy as np
# Step: Create new column 'phone' from formula ''1' + phone_number'
df_all['phone'] = '1' + df_all['phone_number']

# Step: Drop columns
df_all = df_all.drop(columns=['phone_number'])

In [17]:
df_all

,status_name,call_date,phone
0,Agent Lost Connection,2023-06-07 15:29:52,15735288857
1,Call Abandoned In Queue,2023-06-07 15:28:16,19144506875
2,Call Abandoned In Queue,2023-06-07 15:27:27,19297635297
3,Call Abandoned In Queue,2023-06-07 15:26:52,19155887834
4,Queue Drop Call Action Trigger,2023-06-07 15:25:17,17707127186
...,...,...,...
210,Sale,2023-06-07 07:04:19,17042682752
211,Early Hangup,2023-06-07 07:04:10,17042682752
212,Existing Customer,2023-06-07 07:03:30,19033377747
213,Caller Hung Up,2023-06-07 07:02:45,12197027276


In [18]:
import pandas as pd; import numpy as np
# Step: Select columns
PX = PX[['Call Date', 'Target Number', 'Caller ID']]

# Step: Change data type of Target Number to String/Text
PX['Target Number'] = PX['Target Number'].astype('string')

# Step: Rename column
PX = PX.rename(columns={'Caller ID': 'phone'})

merged_df = pd.merge(PX, df_all, on=['phone'], how='left')

# Merged df_all and PX into df3
temp_df = PX.drop_duplicates(subset=['phone']) # Remove duplicates so lookup merge only returns first match
PX_tmp = temp_df.drop(['Target Number', 'Call Date'], axis=1)
df3 = df_all.merge(PX_tmp, left_on=['phone'], right_on=['phone'], how='left', suffixes=['_df_all', '_PX'])


In [19]:
print(df3)

                        status_name           call_date        phone
0             Agent Lost Connection 2023-06-07 15:29:52  15735288857
1           Call Abandoned In Queue 2023-06-07 15:28:16  19144506875
2           Call Abandoned In Queue 2023-06-07 15:27:27  19297635297
3           Call Abandoned In Queue 2023-06-07 15:26:52  19155887834
4    Queue Drop Call Action Trigger 2023-06-07 15:25:17  17707127186
..                              ...                 ...          ...
210                            Sale 2023-06-07 07:04:19  17042682752
211                    Early Hangup 2023-06-07 07:04:10  17042682752
212               Existing Customer 2023-06-07 07:03:30  19033377747
213                  Caller Hung Up 2023-06-07 07:02:45  12197027276
214                        Dead Air 2023-06-07 07:01:36  17864880543

[215 rows x 3 columns]


In [20]:
import os
import smtplib
import mimetypes
from email.mime.multipart import MIMEMultipart
from email import encoders
from email.mime.base import MIMEBase
import pandas as pd

def send_email_with_attachment():
    # Define the CSV file name and DataFrame object
    csv_file = 'merged_data.csv'
    # df3 = pd.DataFrame()  # Replace with your actual DataFrame

    # Save DataFrame to CSV
    df3.to_csv(csv_file, index=False)

    # Define the email sender and recipient
    email_from = "marketing@caridamedsupp.com"
    email_to = ["msavenko@carida.com" ,"accounting@wecallmedia.com"]
    # email_to = ["msavenko@carida.com" ,"msavenko@carida.com"] 

    # Convert the email_to list to a comma-separated string
    email_to_str = ",".join(email_to)
    #email_to_str = email_to

    # Create the email message
    msg = MIMEMultipart()
    msg["From"] = email_from
    msg["To"] = email_to_str
    msg["Subject"] = "Carida Disposition Report for We Call Media"
    msg.preamble = "This is a report of the merged data."

    # Attach the CSV file to the email
    ctype, encoding = mimetypes.guess_type(csv_file)
    if ctype is None or encoding is not None:
        ctype = "application/octet-stream"
    maintype, subtype = ctype.split("/", 1)

    with open(csv_file, "rb") as fp:
        attachment = MIMEBase(maintype, subtype)
        attachment.set_payload(fp.read())
        encoders.encode_base64(attachment)
        attachment.add_header("Content-Disposition", "attachment", filename=csv_file)

    msg.attach(attachment)

    # Send the email
    try:
        smtp = smtplib.SMTP("mail.caridamedsupp.com")
        smtp.login("marketing@caridamedsupp.com", "Carida123")
        smtp.sendmail(email_from, email_to, msg.as_string())
        smtp.quit()
        print("Email sent successfully!")
    except Exception as e:
        print(f"An error occurred while sending the email: {str(e)}")

# Call the function to send the email
send_email_with_attachment()

Email sent successfully!
